<a href="https://colab.research.google.com/github/sanjit1995/NYCTaxiDuration_Kaggle/blob/master/NYC_Taxi_Duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/NYC_Taxi_Duration/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/NYC_Taxi_Duration/test.csv")

In [0]:
train.info()

In [0]:
train['start_time'] = pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.hour
print(train['start_time'])

In [0]:
sns.heatmap(train.isnull(), yticklabels=False, cbar=False)

In [0]:
train['distance'] = abs(train['dropoff_longitude']-train['pickup_longitude']) + abs(train['dropoff_latitude']-train['pickup_latitude'])

In [0]:
train = train.drop(['id','pickup_datetime','dropoff_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1)

In [0]:
train = train[['vendor_id','start_time','passenger_count','distance','store_and_fwd_flag','trip_duration']]

In [0]:
temp_train = train.copy()
train['store_and_fwd_flag'] = train['store_and_fwd_flag'].str[0].str.upper().map({'N':1, 'Y':0})
#temp_train = temp_train.store_and_fwd_flag.replace(['N','Y'],[-1,1])

In [0]:
test['start_time'] = pd.to_datetime(test['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.hour
test['distance'] = abs(test['dropoff_longitude']-test['pickup_longitude']) + abs(test['dropoff_latitude']-test['pickup_latitude'])
test = test.drop(['id','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1)
test = test[['vendor_id','start_time','passenger_count','distance','store_and_fwd_flag']]
test['store_and_fwd_flag'] = test['store_and_fwd_flag'].str[0].str.upper().map({'N':1, 'Y':0})

In [0]:
train

In [0]:
timeOfDay_dict = dict.fromkeys([22, 23, 0, 1, 2, 3, 4, 5], 'Night')
timeOfDay_dict.update(dict.fromkeys([14, 15, 16, 17, 18, 19, 20, 21], 'Afternoon/Evening'))
timeOfDay_dict.update(dict.fromkeys([6, 7, 8, 9, 10, 11, 12, 13], 'Morning'))

In [0]:
train = train.replace({"start_time": timeOfDay_dict})
test = test.replace({"start_time": timeOfDay_dict})

In [0]:
train = pd.concat([train,pd.get_dummies(train['start_time'], drop_first=True)], axis=1)
train = train.drop(['start_time'], axis=1)
test = pd.concat([test,pd.get_dummies(test['start_time'], drop_first=True)], axis=1)
test = test.drop(['start_time'], axis=1)

In [0]:
train

In [0]:
x_train = train.drop(['trip_duration'], axis=1)
y_train = train['trip_duration']

In [0]:
print(x_train.shape, y_train.shape, test.shape)

(1458644, 6) (1458644,) (625134, 6)


In [0]:
n_estimators = [500, 700, 900, 1100, 1300]
max_depth = [2, 3, 5, 10, 15]
booster = ['gbtree','gblinear']
learning_rate = [0.05, 0.1, 0.15, 0.20]
min_child_weight = [1,2,3,4]
base_score = [0.25,0.5,0.75,1]

hyperparameter_grid = {
    'n_estimators' : n_estimators,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'base_score' : base_score
}

import xgboost
regressor = xgboost.XGBRegressor()

from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

random_cv.fit(x_train, y_train)

In [0]:
random_cv.best_estimator_

In [0]:
import xgboost
model = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=900,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
# Fit on training data
model.fit(x_train, y_train)

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, x_train, y_train, cv=5)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   17.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   19.7s finished


array([-1.50695103, -0.29531423, -0.13167782, -0.03324243, -0.21894926])

In [0]:
random_grid = {'bootstrap': [True, False],
               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [130, 180, 230]}
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(verbose=2, n_estimators=60)
#from sklearn.model_selection import RandomizedSearchCV
#rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
model.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.9min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=2, warm_start=False)

In [0]:
rf_random.best_estimator_

In [0]:
y_pred = model.predict(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   39.3s finished


In [0]:
y_pred.shape

(625134,)

In [0]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/NYC_Taxi_Duration/sample_submission.csv")
datasets = pd.concat([sub_df['id'],pred], axis=1)
datasets.columns = ['id', 'trip_duration']
datasets.to_csv('my_submission4.csv', index=False)

In [0]:
datasets.shape

(625134, 2)

In [0]:
#get correlations of each features in dataset
corrmat = train.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(25,25))
#plot heat map
g=sns.heatmap(train[top_corr_features].corr(),annot=True,cmap="RdYlGn")